In [0]:
# some libraries
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
uploaded = files.upload()

Saving sgp_series.csv to sgp_series.csv


In [3]:
# to read the dataframe
import io
df = pd.read_csv(io.StringIO(uploaded['sgp_series.csv'].decode('utf-8')))
df

,Date,SGP_TM_TAX_MANF_BR_ZS,SGP_TM_TAX_TCOM_BR_ZS,SGP_FM_LBL_BMNY_GD_ZS,SGP_FM_LBL_BMNY_CN,SGP_FM_LBL_BMNY_ZG,SGP_FM_LBL_BMNY_IR_ZS,SGP_IQ_WEF_CUST_XQ,SGP_GC_BAL_CASH_GD_ZS,SGP_IC_BUS_DISC_XQ,SGP_GC_BAL_CASH_CN,SGP_GC_DOD_TOTL_CN,SGP_GC_DOD_TOTL_GD_ZS,SGP_NE_GDI_STKB_KN,SGP_NE_GDI_STKB_CD,SGP_NE_GDI_STKB_CN,SGP_NV_MNF_CHEM_ZS_UN,SGP_BN_RES_INCL_CD,SGP_SE_PRM_UNER_FE,SGP_SE_PRM_UNER,SGP_SE_PRM_UNER_MA,SGP_FS_AST_CGOV_GD_ZS,SGP_FM_AST_CGOV_ZG_M3,SGP_FM_AST_PRVT_ZG_M3,SGP_EN_ATM_CO2E_KD_GD,SGP_EN_ATM_CO2E_PP_GD_KD,SGP_EN_ATM_CO2E_PC,SGP_EN_ATM_CO2E_PP_GD,SGP_EN_ATM_CO2E_KT,SGP_EN_CO2_ETOT_ZS,SGP_EN_CO2_ETOT_MT,SGP_EN_ATM_CO2E_GF_ZS,SGP_EN_ATM_CO2E_LF_ZS,SGP_EN_ATM_CO2E_LF_KT,SGP_EN_ATM_CO2E_GF_KT,SGP_EN_CO2_MANF_ZS,SGP_EN_CO2_MANF_MT,SGP_EN_CO2_OTHX_MT,SGP_EN_CO2_BLDG_ZS,SGP_EN_CO2_OTHX_ZS,...,SGP_TM_VAL_AGRI_ZS_UN,SGP_TX_VAL_AGRI_ZS_UN,SGP_EA_PRD_AGRI_KD,SGP_NV_AGR_TOTL_ZS,SGP_NV_AGR_TOTL_KN,SGP_NV_AGR_TOTL_KD_ZG,SGP_NV_AGR_TOTL_KD,SGP_NV_AGR_TOTL_CN,SGP_IS_AIR_GOOD_MT_K1,SGP_NV_AGR_TOTL_CD,SGP_IS_AIR_PSGR,SGP_IS_AIR_DPRT,SGP_EG_USE_COMM_CL_ZS,SGP_ER_H2O_FWAG_ZS,SGP_ER_H2O_FWDM_ZS,SGP_ER_H2O_FWIN_ZS,SGP_ER_H2O_FWTL_ZS,SGP_ER_H2O_FWTL_K3,SGP_AG_LND_ARBL_ZS,SGP_SH_HIV_ARTC_ZS,SGP_AG_LND_ARBL_HA,SGP_AG_LND_ARBL_HA_PC,SGP_MS_MIL_TOTL_TF_ZS,SGP_MS_MIL_XPRT_KD,SGP_MS_MIL_TOTL_P1,SGP_MS_MIL_MPRT_KD,SGP_FB_ATM_TOTL_P5,SGP_AG_LND_PRCP_MM,SGP_FB_BNK_CAPA_ZS,SGP_FB_AST_NPER_ZS,SGP_TM_TAX_MANF_BC_ZS,SGP_TM_TAX_MRCH_BC_ZS,SGP_TM_TAX_TCOM_BC_ZS,SGP_EN_BIR_THRD_NO,SGP_SP_DYN_CBRT_IN,SGP_SH_STA_BRTC_ZS,SGP_FB_CBK_BRWR_P3,SGP_TM_TAX_MRCH_BR_ZS,CPI,CPI_
0,31/12/1995,5.56,9.52,81.852502,1.020000e+11,8.499681,1.045397,NaN,19.012597,NaN,2.368500e+10,9.058500e+10,72.715057,4314100000,6.104134e+08,865200000,9.125567,8.595135e+09,NaN,31471.0,NaN,-29.102880,-8.054905,19.694616,0.412499,0.266376,11.965980,0.365436,42174.167,82.060154,30.87,8.025389,88.096687,37154.044,3384.641,1.250998,4.92,0.10,1.384083,2.661698e-01,...,0.892325,1.082678,34176.75101,0.144732,186400000,-3.719008,1.367070e+08,180300000,3686.899902,1.272047e+08,10779300.00,51600.0000,0.091937,NaN,NaN,NaN,NaN,NaN,1.492537,NaN,1000.000000,0.000284,3.673578,NaN,65500,306000000,NaN,NaN,NaN,NaN,68.73,71.86,83.42,NaN,15.6,NaN,NaN,6.59,9.26,3
1,31/12/1996,5.56,9.48,82.359727,1.120000e+11,9.790326,1.031624,NaN,16.343055,NaN,2.221500e+10,9.875800e+10,72.653946,-1259800000,-2.731206e+09,-3851000000,9.250201,7.394924e+09,NaN,NaN,NaN,-30.097999,-4.567119,17.051428,0.452049,0.291916,13.539324,0.393294,49698.851,84.030122,32.39,6.072456,90.607246,45030.760,3017.941,1.038691,5.31,0.10,1.428200,2.596728e-01,...,0.732279,0.842661,35258.52585,0.149857,192300000,3.165236,1.410341e+08,203700000,4115.000000,1.444681e+08,11840700.00,57200.0000,0.087837,NaN,NaN,NaN,NaN,NaN,1.492537,NaN,1000.000000,0.000272,3.535024,0.0,65500,171000000,NaN,NaN,NaN,NaN,68.43,71.67,83.78,NaN,15.2,99.7,NaN,6.57,8.80,7
2,31/12/1997,5.56,9.48,83.002350,1.230000e+11,10.266099,1.164566,NaN,20.228141,NaN,3.008400e+10,1.060000e+11,71.332372,2867400000,-2.637392e+08,-391600000,10.162514,7.955271e+09,NaN,NaN,NaN,-27.007164,0.666363,14.414342,0.490774,0.316924,15.392359,0.419801,58429.978,83.542977,31.94,5.096021,92.086105,53805.891,2977.604,0.995807,6.76,0.14,1.808176,3.668763e-01,...,0.618494,0.685998,34726.80601,0.132864,189400000,-1.508060,1.389072e+08,197600000,4740.799805,1.330819e+08,12980700.00,60700.0000,0.078799,NaN,NaN,NaN,NaN,NaN,1.492537,NaN,1000.000000,0.000263,9.269361,46000000.0,178000,624000000,NaN,2497.0,NaN,NaN,68.43,71.67,83.78,NaN,14.5,99.7,NaN,6.57,8.66,9
3,31/12/1998,5.56,9.48,112.091232,1.610000e+11,30.248534,1.279626,NaN,10.870027,NaN,1.559200e+10,1.190000e+11,82.619041,-7748400000,-5.595124e+09,-9364000000,13.165626,3.005518e+09,NaN,NaN,NaN,-22.847833,5.988950,9.264930,0.417646,0.269700,12.379539,0.353413,48617.086,83.693305,31.05,6.705385,90.896063,44191.017,3259.963,0.998920,7.24,0.15,1.241901,3.779698e-01,...,0.538740,0.687223,42293.11820,0.111405,173000000,-8.658923,1.268794e+08,159800000,4724.000000,9.548279e+0

In [0]:
import datetime
def datetiem(x):
    date_time_obj = datetime.datetime.strptime(x, '%d/%m/%Y' )
    return date_time_obj

In [0]:
df["Date"] = df["Date"].apply(lambda x: datetiem(x))
df.index = df.Date # index as date because time is not a variable
df=df.drop(['Date','CPI_'], axis=1)

In [9]:
# df0 = df.fillna(0)
# removes columns where more than 30% of data is missing
# rationale for 0.7, if we can train test split for 0.7/ 0.3 we can accept at least 70% filled data
# fills remaining missing values with the mean of the column   
df_mean = df.loc[:, pd.notnull(df).sum()>len(df)*.70].fillna(df.mean())
# df_mean = df.loc[:, pd.notnull(df).sum()>len(df)*.70].fillna(df.interpolate())
df_mean

,SGP_TM_TAX_MANF_BR_ZS,SGP_TM_TAX_TCOM_BR_ZS,SGP_FM_LBL_BMNY_GD_ZS,SGP_FM_LBL_BMNY_CN,SGP_FM_LBL_BMNY_ZG,SGP_FM_LBL_BMNY_IR_ZS,SGP_GC_BAL_CASH_GD_ZS,SGP_GC_BAL_CASH_CN,SGP_GC_DOD_TOTL_CN,SGP_GC_DOD_TOTL_GD_ZS,SGP_NE_GDI_STKB_KN,SGP_NE_GDI_STKB_CD,SGP_NE_GDI_STKB_CN,SGP_NV_MNF_CHEM_ZS_UN,SGP_BN_RES_INCL_CD,SGP_FS_AST_CGOV_GD_ZS,SGP_FM_AST_CGOV_ZG_M3,SGP_FM_AST_PRVT_ZG_M3,SGP_EN_ATM_CO2E_KD_GD,SGP_EN_ATM_CO2E_PP_GD_KD,SGP_EN_ATM_CO2E_PC,SGP_EN_ATM_CO2E_PP_GD,SGP_EN_ATM_CO2E_KT,SGP_EN_CO2_ETOT_ZS,SGP_EN_ATM_CO2E_GF_ZS,SGP_EN_ATM_CO2E_LF_ZS,SGP_EN_ATM_CO2E_LF_KT,SGP_EN_ATM_CO2E_GF_KT,SGP_EN_CO2_MANF_ZS,SGP_EN_CO2_BLDG_ZS,SGP_EN_CO2_OTHX_ZS,SGP_EN_ATM_CO2E_SF_ZS,SGP_EN_ATM_CO2E_SF_KT,SGP_EN_CO2_TRAN_ZS,SGP_NY_GDP_COAL_RT_ZS,SGP_EN_ATM_CO2E_EG_ZS,SGP_EG_USE_CRNW_ZS,SGP_TX_VAL_SERV_CD_WT,SGP_BX_GSR_CMCP_ZS,SGP_TM_VAL_SERV_CD_WT,...,SGP_NY_ADJ_DFOR_GN_ZS,SGP_NY_ADJ_DRES_GN_ZS,SGP_NY_ADJ_DFOR_CD,SGP_NY_ADJ_NNAT_GN_ZS,SGP_NY_ADJ_NNAT_CD,SGP_NY_ADJ_DPEM_GN_ZS,SGP_SP_ADO_TFRT,SGP_NY_ADJ_DPEM_CD,SGP_SP_POP_DPND,SGP_SP_POP_DPND_OL,SGP_SP_POP_DPND_YG,SGP_AG_LND_AGRI_ZS,SGP_AG_LND_AGRI_K2,SGP_TM_VAL_AGRI_ZS_UN,SGP_TX_VAL_AGRI_ZS_UN,SGP_EA_PRD_AGRI_KD,SGP_NV_AGR_TOTL_ZS,SGP_NV_AGR_TOTL_KN,SGP_NV_AGR_TOTL_KD_ZG,SGP_NV_AGR_TOTL_KD,SGP_NV_AGR_TOTL_CN,SGP_IS_AIR_GOOD_MT_K1,SGP_NV_AGR_TOTL_CD,SGP_IS_AIR_PSGR,SGP_IS_AIR_DPRT,SGP_EG_USE_COMM_CL_ZS,SGP_AG_LND_ARBL_ZS,SGP_AG_LND_ARBL_HA,SGP_AG_LND_ARBL_HA_PC,SGP_MS_MIL_TOTL_TF_ZS,SGP_MS_MIL_TOTL_P1,SGP_MS_MIL_MPRT_KD,SGP_FB_AST_NPER_ZS,SGP_TM_TAX_MANF_BC_ZS,SGP_TM_TAX_MRCH_BC_ZS,SGP_TM_TAX_TCOM_BC_ZS,SGP_SP_DYN_CBRT_IN,SGP_SH_STA_BRTC_ZS,SGP_TM_TAX_MRCH_BR_ZS,CPI
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1995-12-31,5.56,9.52,81.852502,1.020000e+11,8.499681,1.045397,19.012597,2.368500e+10,9.058500e+10,72.715057,4314100000,6.104134e+08,865200000,9.125567,8.595135e+09,-29.102880,-8.054905,19.694616,0.412499,0.266376,11.965980,0.365436,42174.16700,82.060154,8.025389,88.096687,37154.0440,3384.6410,1.250998,1.384083,2.661698e-01,0.095644,40.33700,15.011978,0,2.237939,0.000000,2.586958e+10,23.049985,2.170893e+10,...,4.120000e-06,4.120000e-06,3650.490734,37.792620,3.345593e+10,0.100784,7.6202,8.921874e+07,40.048358,8.894099,31.154260,1.791045,12.0,0.892325,1.082678,34176.75101,0.144732,186400000,-3.719008,1.367070e+08,180300000,3686.899902,1.272047e+08,10779300.00,51600.0000,0.091937,1.492537,1000.000000,0.000284,3.673578,65500,306000000,3.052188,68.73,71.86,83.42,15.6,99.7,6.59,9.26
1996-12-31,5.56,9.48,82.359727,1.120000e+11,9.790326,1.031624,16.343055,2.221500e+10,9.875800e+10,72.653946,-1259800000,-2.731206e+09,-3851000000,9.250201,7.394924e+09,-30.097999,-4.567119,17.051428,0.452049,0.291916,13.539324,0.393294,49698.85100,84.030122,6.072456,90.607246,45030.7600,3017.9410,1.038691,1.428200,2.596728e-01,0.000000,0.00000,13.243313,0,2.526271,0.000000,2.701482e+10,24.865738,2.280062e+10,...,0.000000e+00,0.000000e+00,0.000000,36.883562,3.530199e+10,0.100544,7.5306,9.623227e+07,40.820693,9.141922,31.678771,1.791045,12.0,0.732279,0.842661,35258.52585,0.149857,192300000,3.165236,1.410341e+08,203700000,4115.000000,1.444681e+08,11840700.00,57200.0000,0.087837,1.492537,1000.000000,0.000272,3.535024,65500,171000000,3.052188,68.43,71.67,83.78,15.2,99.7,6.57,8.80
1997-12-31,5.56,9.48,83.002350,1.230000e+11,10.266099,1.164566,20.228141,3.008400e+10,1.060000e+11,71.332372,2867400000,-2.637392e+08,-391600000,10.162514,7.955271e+09,-27.007164,0.666363,14.414342,0.490774,0.316924,15.392359,0.419801,58429.97800,83.542977,5.096021,92.086105,53805.8910,2977.6040,0.995807,1.808176,3.668763e-01,0.000000,0.00000,13.259958,0,2.645486,0.000000,2.622760e+10,25.625802,2.290400e+10,...,0.000000e+00,0.000000e+00,0.000000,39.100205,3.988221e+10,0.093290,7.4410,9.515550e+07,41.055041,9.402584,31.652457,1.791045,12.0,0.618494,0.685998,34726.80601,0.132864,189400000,-1.508060,1.389072e+08,197600000,4740.799805,1.330819e+08,12980700.00,60700.0000,0.078799,1.492537,1000.000000,0.000263,9.269361,178

In [0]:
CPI_list = list(df_mean.CPI)
def pct_df(df):
    df.drop(["CPI"], axis=1)
    new_df = df.pct_change()
    new_df.replace(np.nan, 0)
    new_df = new_df.replace([np.inf, -np.inf], 0)
    new_df = new_df.dropna(how='all')
    new_df['CPI']=CPI_list[1:]
    new_df = new_df.fillna(df.mean())
    return new_df
def UpsampleByTimeIndex(df,bytime,method):
    upsampled = df.resample(bytime)
    interpolated = upsampled.interpolate(method=method)
    return interpolated
# Ignore the index and treat the values as equally space
# ‘nearest’, ‘zero’, ‘slinear’, ‘quadratic’, ‘cubic’, ‘spline’, ‘barycentric’, ‘polynomial’
# .agg aggregates/ groups by specified time frame
# D for days, M for months
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.resample.Resampler.aggregate.html 
# https://machinelearningmastery.com/resample-interpolate-time-series-data-python/

In [0]:
pct_df_mean = pct_df(df_mean)

In [0]:
#Ignore this code; helper functions for binarizing
# list_col_names = list(pct_df_mean.columns.values)
from itertools import repeat
def binarize(x):
    if x< 0:
        x = -1
    elif x>0:
        x=1
    else:
        x = 0
    return x
month_cpi = [x for item in CPI_list[1:len(CPI_list)-1] for x in repeat(item, 12)]
month_cpi.insert(len(a),CPI_list[len(CPI_list)-1])
def bin_action_cpi(df):
    df.drop(["CPI"], axis=1)
    df = df.applymap(binarize)
    try:
        df['CPI']=CPI_list[1:]
    except:
        df['CPI']= month_cpi
    return df
def bin_action(df):
    df = df.applymap(binarize)
    return df
def binarize10(x):
    if x<= 0:
        x = 0
    else:
        x = 1
    return x
def bin_action_cpi10(df):
    df.drop(["CPI"], axis=1)
    df = df.applymap(binarize10)
    try:
        df['CPI']=CPI_list[1:]
    except:
        df['CPI']= month_cpi
    return df
def bin_action10(df):
    df = df.applymap(binarize10)
    return df
def binarize123(x):
    if x< 0:
        x = 1
    elif x>0:
        x=3
    else:
        x = 2
    return x
def bin_action_cpi123(df):
    df.drop(["CPI"], axis=1)
    df = df.applymap(binarize123)
    try:
        df['CPI']=CPI_list[1:]
    except:
        df['CPI']= month_cpi
    return df
def bin_action123(df):
    df = df.applymap(binarize123)
    return df

In [0]:
# percentage change df and binarize 123
# df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(pct_df_mean,'Y','linear')
# df_latest = bin_action_cpi123(df_latest)
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'Y','linear')
# df_latest_wo_cpi = bin_action123(df_latest_wo_cpi)
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# percentage change df and binarize 0,1
# df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(pct_df_mean,'M','linear')
# df_latest = bin_action_cpi10(df_latest)
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'M','linear')
# df_latest_wo_cpi = bin_action10(df_latest_wo_cpi)
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# percentage change df and binarize -1,0,1
# df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(pct_df_mean,'Y','linear')
# df_latest = bin_action_cpi(df_latest)
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'Y','linear')
# df_latest_wo_cpi = bin_action(df_latest_wo_cpi)
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# percentage change df
df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
df_latest = UpsampleByTimeIndex(pct_df_mean,'M','linear')
df_latest.index = list(range(len(df_latest)))
df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'M','linear')
df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# mean df
# df_mean_wo_cpi = df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(df_mean,'Y','linear')
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'Y','linear')
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

In [13]:
df_latest_wo_cpi

,SGP_TM_TAX_MANF_BR_ZS,SGP_TM_TAX_TCOM_BR_ZS,SGP_FM_LBL_BMNY_GD_ZS,SGP_FM_LBL_BMNY_CN,SGP_FM_LBL_BMNY_ZG,SGP_FM_LBL_BMNY_IR_ZS,SGP_GC_BAL_CASH_GD_ZS,SGP_GC_BAL_CASH_CN,SGP_GC_DOD_TOTL_CN,SGP_GC_DOD_TOTL_GD_ZS,SGP_NE_GDI_STKB_KN,SGP_NE_GDI_STKB_CD,SGP_NE_GDI_STKB_CN,SGP_NV_MNF_CHEM_ZS_UN,SGP_BN_RES_INCL_CD,SGP_FS_AST_CGOV_GD_ZS,SGP_FM_AST_CGOV_ZG_M3,SGP_FM_AST_PRVT_ZG_M3,SGP_EN_ATM_CO2E_KD_GD,SGP_EN_ATM_CO2E_PP_GD_KD,SGP_EN_ATM_CO2E_PC,SGP_EN_ATM_CO2E_PP_GD,SGP_EN_ATM_CO2E_KT,SGP_EN_CO2_ETOT_ZS,SGP_EN_ATM_CO2E_GF_ZS,SGP_EN_ATM_CO2E_LF_ZS,SGP_EN_ATM_CO2E_LF_KT,SGP_EN_ATM_CO2E_GF_KT,SGP_EN_CO2_MANF_ZS,SGP_EN_CO2_BLDG_ZS,SGP_EN_CO2_OTHX_ZS,SGP_EN_ATM_CO2E_SF_ZS,SGP_EN_ATM_CO2E_SF_KT,SGP_EN_CO2_TRAN_ZS,SGP_NY_GDP_COAL_RT_ZS,SGP_EN_ATM_CO2E_EG_ZS,SGP_EG_USE_CRNW_ZS,SGP_TX_VAL_SERV_CD_WT,SGP_BX_GSR_CMCP_ZS,SGP_TM_VAL_SERV_CD_WT,...,SGP_NY_ADJ_DMIN_CD,SGP_NY_ADJ_DFOR_GN_ZS,SGP_NY_ADJ_DRES_GN_ZS,SGP_NY_ADJ_DFOR_CD,SGP_NY_ADJ_NNAT_GN_ZS,SGP_NY_ADJ_NNAT_CD,SGP_NY_ADJ_DPEM_GN_ZS,SGP_SP_ADO_TFRT,SGP_NY_ADJ_DPEM_CD,SGP_SP_POP_DPND,SGP_SP_POP_DPND_OL,SGP_SP_POP_DPND_YG,SGP_AG_LND_AGRI_ZS,SGP_AG_LND_AGRI_K2,SGP_TM_VAL_AGRI_ZS_UN,SGP_TX_VAL_AGRI_ZS_UN,SGP_EA_PRD_AGRI_KD,SGP_NV_AGR_TOTL_ZS,SGP_NV_AGR_TOTL_KN,SGP_NV_AGR_TOTL_KD_ZG,SGP_NV_AGR_TOTL_KD,SGP_NV_AGR_TOTL_CN,SGP_IS_AIR_GOOD_MT_K1,SGP_NV_AGR_TOTL_CD,SGP_IS_AIR_PSGR,SGP_IS_AIR_DPRT,SGP_EG_USE_COMM_CL_ZS,SGP_AG_LND_ARBL_ZS,SGP_AG_LND_ARBL_HA,SGP_AG_LND_ARBL_HA_PC,SGP_MS_MIL_TOTL_TF_ZS,SGP_MS_MIL_TOTL_P1,SGP_MS_MIL_MPRT_KD,SGP_FB_AST_NPER_ZS,SGP_TM_TAX_MANF_BC_ZS,SGP_TM_TAX_MRCH_BC_ZS,SGP_TM_TAX_TCOM_BC_ZS,SGP_SP_DYN_CBRT_IN,SGP_SH_STA_BRTC_ZS,SGP_TM_TAX_MRCH_BR_ZS
0,0.000000,-0.004202,0.006197,0.098039,0.151846,-0.013175,-0.140409,-0.062065,0.090225,-0.000840,-1.292019,-5.474354,-5.450994,1.365777e-02,-0.139638,0.034193,-0.433002,-0.134209,0.095878,0.095878,0.131485,0.076231,0.178419,0.024006,-0.243344,0.028498,0.212002,-0.108342,-0.169710,0.031875,-2.440924e-02,-1.000000,-1.000000,-0.117817,0.0,0.128839,0.003658,0.044270,0.078775,0.050287,...,0.0,-1.000000,-1.000000,-1.000000,-0.024054,0.055179,-0.002381,-0.011758,0.078611,0.019285,0.027864,0.016836,0.0,0.0,-0.179359,-0.221688,0.031652,0.035414,0.031652,-1.851097,0.031652,0.129784,0.116114,0.135713,0.098467,0.108527,-0.044596,0.0,0.0,-0.039830,-0.037716,0.000000,-0.441176,0.000000,-0.004365,-0.002644,0.004316,-0.025641,-1.110223e-16,-0.003035
1,0.000000,-0.003852,0.006331,0.098054,0.143242,-0.001338,-0.108898,-0.027374,0.088817,-0.002286,-1.457357,-5.093444,-5.071604,2.073847e-02,-0.121687,0.022786,-0.492410,-0.135913,0.095027,0.095027,0.131933,0.075495,0.178191,0.021523,-0.236465,0.027483,0.210574,-0.100428,-0.159008,0.051390,1.202825e-02,-0.891571,12.238696,-0.107894,0.0,0.122034,0.003658,0.038152,0.074757,0.046475,...,0.0,-0.916667,-0.916667,79.514110,-0.017041,0.061392,-0.008195,-0.011770,0.071127,0.018156,0.027918,0.015364,0.0,0.0,-0.177361,-0.218707,0.027758,0.023013,0.027758,-1.819876,0.027758,0.116473,0.119111,0.117836,0.098284,0.104582,-0.049454,0.0,0.0,-0.039262,0.100606,0.143130,-0.183652,0.000000,-0.004001,-0.002424,0.003956,-0.027342,-1.017704e-16,-0.002782
2,0.000000,-0.003501,0.006464,0.098068,0.134638,0.010498,-0.077387,0.007316,0.087409,-0.003732,-1.622695,-4.712534,-4.692214,2.781918e-02,-0.103736,0.011379,-0.551819,-0.137616,0.094176,0.094176,0.132381,0.074759,0.177963,0.019039,-0.229586,0.026468,0.209146,-0.092513,-0.148306,0.070904,4.846574e-02,-0.783141,25.477392,-0.097971,0.0,0.115230,0.003658,0.032035,0.070740,0.042662,...,0.0,-0.833333,-0.833333,160.028220,-0.010028,0.067606,-0.014009,-0.011782,0.063644,0.017028,0.027972,0.013892,0.0,0.0,-0.175363,-0.215725,0.023864,0.010612,0.023864,-1.788655,0.023864,0.103162,0.122108,0.099958,0.098102,0.100637,-0.054313,0.0,0.0,-0.038694,0.238928,0.286260,0.073873,0.000000,-0.003637,-0.002203,0.003596,-0.029043,-9.251859e-17,-0.002529
3,0.000000,-0.003151,0.006598,0.098083,0.126034,0.022335,-0.045877,0.042007,0.086001,-0.005178,-1.788033,-4.331624,-4.312824,3.489988e-02,-0.085

In [14]:
# check for null, if false then no missing values
def null_check(dfr):
    return dfr.isnull().values.any()
# print(null_check(df0))
# print(null_check(df_mean))
print(null_check(df_latest_wo_cpi))

False


In [15]:
from sklearn.preprocessing import MinMaxScaler
sc= MinMaxScaler(feature_range=(0,1))

input_data = df_latest.values
test_data = df_latest_wo_cpi.values
input_data[:,0:len(input_data[0])-1] = sc.fit_transform(input_data[:,0:len(input_data[0])-1])
test_data[:,0:len(test_data[0])] = sc.fit_transform(test_data[:,:])
test_data

array([[0.4174641 , 0.25219423, 0.28784186, ..., 0.54088922, 0.49924887,
        0.33803899],
       [0.4174641 , 0.25234104, 0.28811868, ..., 0.53275056, 0.49924887,
        0.33823259],
       [0.4174641 , 0.25248785, 0.28839551, ..., 0.52461191, 0.49924887,
        0.33842619],
       ...,
       [0.34788675, 0.21162996, 0.35648138, ..., 0.53211954, 0.47142937,
        0.28363518],
       [0.38267542, 0.23279296, 0.37319031, ..., 0.52385642, 0.48533912,
        0.3119987 ],
       [0.4174641 , 0.25395595, 0.38989924, ..., 0.51559331, 0.49924887,
        0.34036222]])

In [16]:
print(test_data.shape)
print(input_data.shape) # check data and shape, are there 22/ 229 rows?

(241, 581)
(241, 582)


In [17]:
df_latest.head()

,SGP_TM_TAX_MANF_BR_ZS,SGP_TM_TAX_TCOM_BR_ZS,SGP_FM_LBL_BMNY_GD_ZS,SGP_FM_LBL_BMNY_CN,SGP_FM_LBL_BMNY_ZG,SGP_FM_LBL_BMNY_IR_ZS,SGP_GC_BAL_CASH_GD_ZS,SGP_GC_BAL_CASH_CN,SGP_GC_DOD_TOTL_CN,SGP_GC_DOD_TOTL_GD_ZS,SGP_NE_GDI_STKB_KN,SGP_NE_GDI_STKB_CD,SGP_NE_GDI_STKB_CN,SGP_NV_MNF_CHEM_ZS_UN,SGP_BN_RES_INCL_CD,SGP_FS_AST_CGOV_GD_ZS,SGP_FM_AST_CGOV_ZG_M3,SGP_FM_AST_PRVT_ZG_M3,SGP_EN_ATM_CO2E_KD_GD,SGP_EN_ATM_CO2E_PP_GD_KD,SGP_EN_ATM_CO2E_PC,SGP_EN_ATM_CO2E_PP_GD,SGP_EN_ATM_CO2E_KT,SGP_EN_CO2_ETOT_ZS,SGP_EN_ATM_CO2E_GF_ZS,SGP_EN_ATM_CO2E_LF_ZS,SGP_EN_ATM_CO2E_LF_KT,SGP_EN_ATM_CO2E_GF_KT,SGP_EN_CO2_MANF_ZS,SGP_EN_CO2_BLDG_ZS,SGP_EN_CO2_OTHX_ZS,SGP_EN_ATM_CO2E_SF_ZS,SGP_EN_ATM_CO2E_SF_KT,SGP_EN_CO2_TRAN_ZS,SGP_NY_GDP_COAL_RT_ZS,SGP_EN_ATM_CO2E_EG_ZS,SGP_EG_USE_CRNW_ZS,SGP_TX_VAL_SERV_CD_WT,SGP_BX_GSR_CMCP_ZS,SGP_TM_VAL_SERV_CD_WT,...,SGP_NY_ADJ_DFOR_GN_ZS,SGP_NY_ADJ_DRES_GN_ZS,SGP_NY_ADJ_DFOR_CD,SGP_NY_ADJ_NNAT_GN_ZS,SGP_NY_ADJ_NNAT_CD,SGP_NY_ADJ_DPEM_GN_ZS,SGP_SP_ADO_TFRT,SGP_NY_ADJ_DPEM_CD,SGP_SP_POP_DPND,SGP_SP_POP_DPND_OL,SGP_SP_POP_DPND_YG,SGP_AG_LND_AGRI_ZS,SGP_AG_LND_AGRI_K2,SGP_TM_VAL_AGRI_ZS_UN,SGP_TX_VAL_AGRI_ZS_UN,SGP_EA_PRD_AGRI_KD,SGP_NV_AGR_TOTL_ZS,SGP_NV_AGR_TOTL_KN,SGP_NV_AGR_TOTL_KD_ZG,SGP_NV_AGR_TOTL_KD,SGP_NV_AGR_TOTL_CN,SGP_IS_AIR_GOOD_MT_K1,SGP_NV_AGR_TOTL_CD,SGP_IS_AIR_PSGR,SGP_IS_AIR_DPRT,SGP_EG_USE_COMM_CL_ZS,SGP_AG_LND_ARBL_ZS,SGP_AG_LND_ARBL_HA,SGP_AG_LND_ARBL_HA_PC,SGP_MS_MIL_TOTL_TF_ZS,SGP_MS_MIL_TOTL_P1,SGP_MS_MIL_MPRT_KD,SGP_FB_AST_NPER_ZS,SGP_TM_TAX_MANF_BC_ZS,SGP_TM_TAX_MRCH_BC_ZS,SGP_TM_TAX_TCOM_BC_ZS,SGP_SP_DYN_CBRT_IN,SGP_SH_STA_BRTC_ZS,SGP_TM_TAX_MRCH_BR_ZS,CPI
0,0.417464,0.252194,0.287842,0.353421,0.860269,0.332317,0.142590,0.136836,0.442800,0.332010,0.621817,0.000000,0.000000,0.543805,0.364374,0.705543,0.306311,0.336531,0.423125,0.423125,0.464948,0.421052,0.455616,0.413506,0.107843,0.314869,0.241008,0.344800,0.041461,0.595707,1.0,0.000000,0.000000,0.000000,0.0,0.466295,0.618574,0.416098,0.867741,0.439689,...,0.000000,0.000000,0.000000,0.477530,0.411903,0.623216,0.852140,0.557620,1.000000,0.380270,1.000000,0.841184,0.833333,0.015114,0.000000,0.140865,1.000000,0.832137,0.001581,0.832137,1.000000,0.776338,1.000000,0.525209,0.325129,0.207408,0.715421,0.705882,0.646596,0.061646,0.063656,0.098614,0.255491,0.380488,0.412034,0.190534,0.540889,0.499249,0.338039,8.800000
1,0.417464,0.252341,0.288119,0.353466,0.859978,0.387138,0.149460,0.143371,0.437927,0.328158,0.612153,0.014828,0.013376,0.554904,0.366958,0.699454,0.301417,0.336313,0.422409,0.422409,0.465356,0.420430,0.455420,0.405999,0.111703,0.314298,0.240630,0.351893,0.042819,0.629398,1.0,0.015627,0.083333,0.049156,0.0,0.460860,0.618574,0.401951,0.852033,0.429324,...,0.083333,0.083333,0.083333,0.494496,0.419590,0.600643,0.852021,0.530756,0.970792,0.381277,0.970300,0.841184,0.833333,0.019083,0.005171,0.138160,0.940192,0.816162,0.002027,0.816162,0.958544,0.788134,0.957256,0.524863,0.319390,0.201863,0.715421,0.705882,0.647641,0.139842,0.141685,0.152035,0.255491,0.387945,0.418415,0.176585,0.532751,0.499249,0.338233,8.788333
2,0.417464,0.252488,0.288396,0.353511,0.859686,0.441959,0.156329,0.149906,0.433053,0.324307,0.602488,0.029656,0.026752,0.566003,0.369543,0.693366,0.296523,0.336095,0.421693,0.421693,0.465764,0.419809,0.455225,0.398492,0.115563,0.313727,0.240251,0.358986,0.044177,0.663089,1.0,0.031255,0.166667,0.098312,0.0,0.455425,0.618574,0.387803,0.836325,0.418959,...,0.166667,0.166667,0.166667,0.511461,0.427276,0.578070,0.851902,0.503893,0.941584,0.382284,0.940599,0.841184,0.833333,0.023053,0.010342,0.135456,0.880384,0.800188,0.002472,0.800188,0.917087,0.799930,0.914512,0.524518,0.313651,0.196318,0.715421,0.705882,0.648687,0.218038,0.219714,0.205456,0.255491,0.395403,0.424796,0.162636,0.524612,0.499249,0.338426,8.776667
3,0.417464,0.252635,0.288672,0.353556,0.859395,0.496780,0.163199,0.156440,0.428180,0.320456,0.592824,0.044483,0.040128,0.577102,0.372127,0.687277,0.291629,0.335877,0.420977,0.420977,0.466172,0.419187,0.455029,0.390985,0.119424,0.313156,0

In [18]:
n_steps = 3
df_test=test_data[-n_steps:]
CPI_test = df_latest['CPI'][-n_steps:]
print(CPI_test)
print(df_test.shape)
input_feature = input_data[:len(input_data)-n_steps] # to remove last 5 rows, follow the time step you set

238    84.166667
239    84.083333
240    84.000000
Name: CPI, dtype: float64
(3, 581)


In [19]:
input_feature.shape # check shape 

(238, 582)

In [0]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [21]:
# choose a number of time steps
from numpy import array
# convert into input/output
X, y = split_sequences(input_feature, n_steps)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])
# the array is (Total number of samples, time step, total number of dimensions)

(236, 3, 581) (236,)
[[0.4174641  0.25219423 0.28784186 ... 0.54088922 0.49924887 0.33803899]
 [0.4174641  0.25234104 0.28811868 ... 0.53275056 0.49924887 0.33823259]
 [0.4174641  0.25248785 0.28839551 ... 0.52461191 0.49924887 0.33842619]] 8.776666666666667
[[0.4174641  0.25234104 0.28811868 ... 0.53275056 0.49924887 0.33823259]
 [0.4174641  0.25248785 0.28839551 ... 0.52461191 0.49924887 0.33842619]
 [0.4174641  0.25263466 0.28867233 ... 0.51647326 0.49924887 0.33861979]] 8.765
[[0.4174641  0.25248785 0.28839551 ... 0.52461191 0.49924887 0.33842619]
 [0.4174641  0.25263466 0.28867233 ... 0.51647326 0.49924887 0.33861979]
 [0.4174641  0.25278147 0.28894916 ... 0.50833461 0.49924887 0.3388134 ]] 8.753333333333334
[[0.4174641  0.25263466 0.28867233 ... 0.51647326 0.49924887 0.33861979]
 [0.4174641  0.25278147 0.28894916 ... 0.50833461 0.49924887 0.3388134 ]
 [0.4174641  0.25292828 0.28922598 ... 0.50019595 0.49924887 0.339007  ]] 8.741666666666667
[[0.4174641  0.25278147 0.28894916 ... 

In [22]:
# final model
# https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/ 
# refer to this if necessary
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
model = Sequential()
model.add(LSTM(75, activation='relu', input_shape=(n_steps, X.shape[2])))
model.add(Dense(15))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# model.save_weights('my_model_weights.h5')

Using TensorFlow backend.


In [23]:
# fit model
model.fit(X, y, epochs=200, verbose=0)#, validation_split=0.2, batch_size=5)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










In [0]:
# demonstrate prediction
x_input = df_test
x_input = x_input.reshape((1, n_steps,X.shape[2] ))
yhat = model.predict(x_input, verbose=0)

Percentage Error

In [25]:
print(yhat)
actual = CPI_test[len(df_latest)-1]
print(float((abs(actual-yhat)/actual)*100))

[[83.593376]]
0.48407599329948425


In [0]:
weight_tensors = model.layers[1].weights
weight_names = list(map(lambda x: x.name, weight_tensors))
print(weight_names)

['dense_10/kernel:0', 'dense_10/bias:0']


In [0]:
for layer in model.layers:
        if "LSTM" in str(layer):
            weightLSTM = layer.get_weights()
warr,uarr, barr = weightLSTM
print(warr.shape) # warr is a numpy array of weights for inputs
print(uarr.shape) # uarr is a numpy array of weights for hidden units
print(barr.shape) # barr is a numpy array of bias

(674, 300)
(75, 300)
(300,)


In [0]:
print(warr)
print(uarr)
print(barr)

In [0]:
# change df to percentage changes
# play with the rank/ totaln
# interpolate lesser 
# time step

In [0]:
time step = 5, CPI, indicator data
interpolate by year, 22 data points
# 20.441774368286133
interpolate by month, 262 DP
# 1.0591048002243042
interpolate by day, 7920 DP
# 0.015229816548526287 

After performing percentage change and normalizing 
interpolate by year time step 7
# 0.6501209735870361
# 351430.6875
interpolate by year time step 5
# 0.5464288592338562
# 320606.21875
interpolate by year time step 3
# 1.6997615098953247
interpolate by month time step 7
# 1.4450266361236572
interpolate by month time step 5
# 0.217756450176239


*** champion ***
interpolate by month time step 3
# 0.4332916736602783 25 units
# 0.15020249783992767 50 units 0.24396678805351257
# 0.08212704956531525 75 units
# 0.025785423815250397 75 units dense 15 , 1
# 0.09989678114652634 75 units dense 25 , 1
***  winner  # 0.016958503052592278 75 units dense 35 , 1 
*** end ***


binarize -1,0,1
year time step = 7
# 63.047935485839844
year time step = 5
# 59.16572189331055
year time step = 3
# 82.13848114013672
month time step = 7
# 11.621817588806152
month time step = 5
# 11.747268676757812
month time step = 3
# 15.268765449523926


binarize 0,1
year time step = 7
# 1.3483071327209473
year time step = 5
# 2.037656784057617
year time step = 3
# 4.179565906524658
month time step = 7
# 0.44504478573799133
month time step = 5
# 4.9152421951293945
month time step = 3
# 3.9173173904418945

binarize 1,2,3
year time step = 7
# 5.180117607116699
year time step = 5
# 4.566781520843506
year time step = 3
# 1.8392828702926636
month time step = 7
# 3.652085065841675
month time step = 5
# 4.432069778442383
month time step = 3
# 0.7650640606880188